## Sun B1 dynamics: nastier than expected...
When we have 3BP dynamics in E-M rotating frame, we naturally forget about $\omega$ as we normalize t* so that $\omega$ = 1 (i.e., $t_1^*$ = 2$\pi$ / (1 siderial period = 27.5days). 
This means omega should change if we want to discuss the dynamics in Sun-B1 frame ($t_2^*$ = 2$\pi$ / 365 days), while we want to stick with $t_1^*$. Therefore, the angular velocity term in the centrifugal and Coriolis force emerges: $\omega_b = 2\pi$ / (365 days / 27.5 days).

Additionally, according to Boudad's PhD thesis (C.19, P.235), the angular velocity of the E-M line should be $\omega_M = |\omega_S|/(1-|\omega_S|) ~= 1.05$, rather than $\omega_S = - t_{siderial}/t_{synodic} ~= -0.93$, which is the angular velocity of the Sun w.r.t. Earth-Moon rotating frame in BCR4P.




### normalization
$t^* = t_{syderial} = 27.5 * 86400$ [s], 

$l^*$ = Earth-Moon distance,

$m^* = m_E + m_M$.

In [7]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path:\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.\n")
└ @ Plots C:\Users\yujit\.julia\packages\Plots\gzYVM\src\backends.jl:420


Plots.PlotlyBackend()

In [8]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")
param3b = SailorMoon.dyanmics_parameters()


Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 12.36898385069774, 0.01709689063726318, 7.601281331451572)

In [33]:
"""
Right-hand side expression for state-vector in BCR4BP
# Arguments
    - `du`: cache array of duative of state-vector, mutated
    - `u`: state-vector
    - `p`: parameters, where p = [μ2, μS, as, θ0, ωM, τ, γ, β, mdot, tmax]
        m* : mE + mL (6.0455 * 10^22)
        μ2 : mL / m* (0.012150585609624)
        μS : mS / m* (0.00000303951)
        as : (sun-B1 distance) / l* (388.709677419)
            l* : Earth-moon distance (384,400 km)
        ωM : Earth-Moon line's angular velocity around E-M barycenter
        τ  : thrust magnitude (0~1)
        γ  : thrust angle 1
        β  : thrust angle 2
        mdot : mass-flow rate
        tmax : max thrust
    - `t`: time
"""
function rhs_bcr4bp_sb1frame!(du,u,p,t)
    # unpack state
    x, y, z = u[1], u[2], u[3]
    vx, vy, vz = u[4], u[5], u[6]
    μ2, μS, as, θ0, ωM, ωb = p[1], p[2], p[3], p[4], p[5], p[6]
    τ, γ, β, mdot, tmax = p[7], p[8], p[9], p[10], p[11]
    
    ωM = 0.9251999994040079

    θ = θ0 + ωM * t  # moon angle

    # create Thrust term
#     T = dv_inertial_angles([τ,γ,β])
    T = [0.0, 0.0, 0.0]
    Tx, Ty, Tz = T * tmax / u[7]  # mdot


    # derivatives of positions
    du[1] = u[4]
    du[2] = u[5]
    du[3] = u[6]

    # earth and moon location
    xe = - μ2 *cos(θ) 
    ye = - μ2 *sin(θ)
    ze = 0

    xm = (1-μ2) *cos(θ) 
    ym = (1-μ2) *sin(θ)
    zm = 0
    
#     println(xe, " ", ye, " ", ze)

    # compute distances
    r30 = sqrt((as + x)^2 + y^2 + z^2)      # SC-Sun
    r31 = sqrt((xe - x)^2 + (ye - y)^2 + (ze-z)^2)  # SC-earth
    r32 = sqrt((xm - x)^2 + (ym - y)^2 + (zm-z)^2)  # SC-Moon
    println(r30, " ", r31, " ", r32)

    Fx = -(μS)*(x+as)/r30^3 - (1-μ2)*(x-xe)/r31^3 - μ2*(x-xm)/r32^3 + Tx
    Fy = -(μS)* y / r30^3   - (1-μ2)*(y-ye)/r31^3 - μ2*(y-ym)/r32^3 + Ty
    Fz = -(μS)* z / r30^3   - (1-μ2)*(z-ze)/r31^3 - μ2*(z-zm)/r32^3 + Tz
    println(Fx, " ", Fy, " ", Fz)
    # println(-(μS/(μS+1))*(x-1/(μS+1))/r30^3 , " ", - (1-μ2)/(μS+1)*(x-xe)/r31^3, " ",  - μ2/(μS+1)*(x-xm)/r32^3)
#     println(Fx, " ", Fy, " ", Fz)

    du[4] =  2*ωb*vy + ωb^2*x + Fx
    du[5] = -2*ωb*vx + ωb^2*y + Fy
    du[6] =                   + Fz

    du[7] = -mdot * τ
end

rhs_bcr4bp_sb1frame!

In [34]:
"""
    transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)

Transform state from Earth-Moon rotating frame to Sun-B1 rotating frame.
Careful with sign of ωs!! (should be negative)
"""
function transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)
    ωm = -ωs
    θm = π - θs
    cos_θm = cos(θm)
    sin_θm = sin(θm)
    C = [
        cos_θm      -sin_θm   0 0       0      0
        sin_θm      cos_θm    0 0       0      0
        0           0         1 0       0      0
        -ωm*sin_θm -ωm*cos_θm 0 cos_θm -sin_θm 0 
         ωm*cos_θm -ωm*sin_θm 0 sin_θm  cos_θm 0
         0          0         0 0       0      1
    ]
    state_conv = C * state    
    return state_conv
end


transform_EMrot_to_SunB1

In [35]:
function plot_circle(radius, x, y, n=50)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [36]:
θM = 180 / 180 * pi  
ϕ0 = 0.0 # 5.27787565803085   #0.0
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
ωb = 2*pi*(t_synodic - t_sidereal)/t_synodic/t_sidereal*param3b.tstar
print(ωb)
param = [param3b.mu2, param3b.mus, param3b.as, θM, param3b.oml, ωb, 0.0, 0.0, 0.0, 0.0, 0.0]

0.07480000059599223

11-element Vector{Float64}:
      0.01215058560962404
 328900.5598102475
    388.8212386592645
      3.141592653589793
     12.36898385069774
      0.07480000059599223
      0.0
      0.0
      0.0
      0.0
      0.0

In [37]:
# initial condition: Halo orbit progapation
lps = SailorMoon.lagrange_points(param3b.mu2)

## set up of initial condition (Lyapunov orbit)
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1) # monodromy eigenvector

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO2(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, 1e-6, Tsit5(), 1e-12, 1e-12
);

# xf in the Earth-moon rotating frame
xf_em = SailorMoon.set_terminal_state2(ϕ0, θf, param3b, LPOArrival)


Halo guess Az_km: 1200.0
Linear stability ν = 618.7618470919092


6-element Vector{Float64}:
  1.1226882532375977
  2.7938137461526135e-7
 -5.228883154550508e-26
 -7.710131455619899e-7
  0.16509880733691906
  1.3395041586794773e-31

In [38]:
# change coordination from EM rot. frame to Sun-B1 frame
θs = π - θM
ωs = param3b.oms
xf_sb1 = vcat(transform_EMrot_to_SunB1(xf_em, θs, ωs), 1.0)
println(xf_sb1)

# propagate ODE
tspan = [0.0, -29]
prob = ODEProblem(rhs_bcr4bp_sb1frame!, xf_sb1, tspan, param)
sol = solve(prob, Tsit5(), reltol=1e-12, abstol=1e-12)

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
plot!(pcart, Array(sol)[1,:], Array(sol)[2,:], color=:blue, linewidth=1.0, label="sol", linestyle=:solid)

moon = plot_circle(1-param3b.mu2, 0.0 , 0.0)
earth = plot_circle(param3b.mu2, 0.0, 0.0)
moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, 0.0, 0.0)


plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

[-1.1226882532375977, -2.793813744777717e-7, -5.228883154550508e-26, 1.0294967930420964e-6, -1.203809978563231, 1.3395041586794773e-31, 1.0]
387.6985504060269 1.1348388388472561 0.13483883884751113
-0.7528046756673985 1.5750938997621411e-6 2.9479352270947036e-25
387.6985504060269 1.1348388388472561 0.13483883884751113
-0.7528046756673985 1.5750938997621411e-6 2.9479352270947036e-25
387.6985504662755 1.134861764721788 0.13486492564833785
-0.7531645833409895 -0.013490759053303679 2.9464102954043115e-25
387.69855040588504 1.1348388914025955 0.13483889854242148
-0.7528054984594895 -0.0006434475704177348 2.947931738195448e-25
387.6985502472953 1.134839213973233 0.13483924360518548
-0.7528098587023763 -0.0013084953140979324 2.947908733748143e-25
387.69854922574217 1.13484165917438 0.13484188446439052
-0.7528437134366325 -0.003603965127418848 2.947736064799502e-25
387.6985490100659 1.1348421795162955 0.13484244669311743
-0.7528509253588592 -0.0039245190251885696 2.9476993360981783e-25
387.698

-0.7833997656246793 -0.09862381357796322 2.793222310469632e-25
387.69758503202326 1.137128872077457 0.13730284948362284
-0.7835108800611869 -0.09879472040211515 2.792666002767258e-25
387.6975849783356 1.1371289272295286 0.13730290593551678
-0.7835114949375316 -0.09879480822848134 2.792661211130683e-25
387.6975552140505 1.137199509183205 0.1373785920069085
-0.7844281800916261 -0.10019524984800793 2.7880760714575745e-25
387.6975235523443 1.1372738958704445 0.13745830129831563
-0.7853911901480723 -0.10163300428377067 2.783254059027917e-25
387.69741132540395 1.137538615436748 0.13774188957217146
-0.7888048120818367 -0.10655609325449006 2.7662011401402994e-25
387.6973951930589 1.1375766620839571 0.13778263008375138
-0.7892934574049167 -0.10723841186414881 2.7637637365962537e-25
387.6973911467785 1.137586201802476 0.1377928444263954
-0.7894158973567703 -0.10740851691795425 2.7631530792830316e-25
387.69739109310945 1.1375862570895006 0.13779290112879627
-0.789416509537521 -0.1074085983906529 

-0.9163225953644037 -0.2057901539810015 2.158388607540435e-25
387.69236010800796 1.14918525301518 0.15002274144528235
-0.9184648415914347 -0.2067783524899184 2.1487031687217665e-25
387.6922577783987 1.1494157381412273 0.15026236683074257
-0.9206865536686801 -0.20778346845791576 2.138669737315365e-25
387.6918992250472 1.1502239810674266 0.15110188138711925
-0.9283916234710939 -0.21117316038023498 2.1040433740771096e-25
387.6918483042978 1.150338600131721 0.15122082310469245
-0.9294732723254783 -0.21163615217596057 2.0992026368051495e-25
387.6918355595381 1.1503672781834213 0.15125057827933458
-0.9297434773457164 -0.21175131409398928 2.0979940721199336e-25
387.6918355094448 1.1503673304110267 0.15125063164271077
-0.9297439183622428 -0.21175126059905144 2.0979893111850954e-25
387.6917306586758 1.150603717572219 0.15149586617349664
-0.9319655579710111 -0.21269362645090847 2.08806577708149e-25
387.6916207086524 1.1508502219909862 0.15175140542182997
-0.934268459499909 -0.21365113140726683 2

387.6778814104234 1.1804259198932698 0.18174126066829502
-1.1449601267887113 -0.261366539446201 1.2398534040646805e-25
387.6769916884533 1.1822668844203785 0.18357680030120402
-1.1549584444944767 -0.262241573860827 1.2055594294257388e-25
387.6768657376232 1.1825268856525393 0.1838358382142859
-1.1563491500126 -0.2623561468643598 1.2008324850768004e-25
387.6768342307928 1.1825918968612037 0.1839006008845193
-1.1566960687903938 -0.2623844466518998 1.1996548978320885e-25
387.6768341489554 1.182591971407181 0.18390067031143623
-1.156696377797524 -0.26238422374247483 1.1996513509189878e-25
387.67657748089215 1.1831221930356415 0.18442879541837948
-1.1595145869290755 -0.2626125830045526 1.19010964839154e-25
387.6763092363224 1.1836736212201517 0.18497775139413533
-1.1624212319406657 -0.2628357039398516 1.180308643811521e-25
387.6753723043006 1.1855980177123575 0.1868920982624654
-1.17239266744601 -0.26355473231868837 1.1470417944981725e-25
387.67523970403005 1.1858697249736303 0.187162196623

387.64569308997727 1.2431630634852966 0.24364554340087902
-1.3864669357607389 -0.26296293719427777 5.668309152813641e-26
387.64563402639675 1.2432722931157307 0.24375310914536752
-1.3867683388112046 -0.2629452787104677 5.661956691846121e-26
387.6456338317631 1.2432724583449515 0.24375326712827616
-1.3867686936459616 -0.2629449697343406 5.661940315614826e-26
387.64515284921083 1.2441624482430769 0.24462977992320661
-1.3892147802555939 -0.2628018072568087 5.610569331994077e-26
387.6446516304873 1.245086118713374 0.24553954580184806
-1.3917331661549794 -0.26264831699836333 5.558010892075134e-26
387.6429054202731 1.248298922076094 0.24870510031552723
-1.400347353123442 -0.26210950196790955 5.380873064277673e-26
387.64265900688565 1.2487511275481884 0.24915079453469755
-1.4015418362290812 -0.2620324233410124 5.356631377106886e-26
387.64259739566444 1.2488641400624794 0.24926218523446528
-1.40183967038403 -0.26201310306930675 5.350598830067088e-26
387.6425971905151 1.2488643137984792 0.24926

...

0.8534828668222094 4.085851744994188 -2.620274167891916e-27
282.48550091043336 526.3246195561472 525.328579168122
0.863555183487063 4.030184115912084 -2.4998775823260922e-27
282.73486528285014 526.6424944628037 525.6470464859563
0.8648982734976983 4.022463128616842 -2.4833421220819174e-27
282.7971463896309 526.7218301780204 525.7265364948711
0.8652311001498814 4.020538037813813 -2.479225270049446e-27
282.79710146264875 526.7217912494573 525.7264975617207
0.8652312208281215 4.020539348462401 -2.4792198232509692e-27
283.3031309229305 527.3659648471879 526.3720189397167
0.8679049337454517 4.004945089279996 -2.4459522165649385e-27
283.8264146576897 528.0298368254935 527.0374577258424
0.870584418062724 3.9889129890009047 -2.4119203652782163e-27
285.6402618993239 530.3205483347762 529.3349544561379
0.8793570933805936 3.934041515117325 -2.2967085432203076e-27
285.8946290115245 530.6403417338057 529.6558649710291
0.8805218029314371 3.9264332690210257 -2.2808911551013006e-27
285.95815855939884 

351.8658006715472 605.1482280004362 605.2866559399826
0.8855648931567996 2.5045559684082663 1.4231760506276824e-29
351.865850842161 605.1482161446016 605.2866442451691
0.8855640322929629 2.504555469280816 1.4232984676508624e-29
352.53748497282413 605.8444323475047 605.9972135449364
0.8840861348847014 2.4943513081204456 2.5799405919293133e-29
353.2314831020622 606.562033423993 606.7295829682924
0.8825363096662312 2.4838779744324593 3.75862826359701e-29
355.63255319536194 609.0386601764229 609.2568835682068
0.8770931911105854 2.44815575140063 7.720966455744258e-29
355.96869632653977 609.3844684269316 609.6097267652123
0.8763196807993968 2.4432180830867893 8.26141167734605e-29
356.0526311071553 609.4707791080018 609.6977916097045
0.8761260787024625 2.4419875615337414 8.395822208399872e-29
356.0526866587137 609.4707698676643 609.6977825368317
0.8761251924404599 2.4419870194181432 8.395929376518312e-29
356.7344559479296 610.1716916214597 610.4129210973872
0.8745505960785578 2.43202586980334

421.7849384881271 673.5566225169397 674.5566080084017
0.7023714383063131 1.7101536049213009 6.779589835856282e-28
421.78508097044437 673.5566626585628 674.5566481484003
0.7023703538351538 1.710152700021353 6.779584464422313e-28
422.62597490229473 674.3398223861252 675.3395814360589
0.7001224295202457 1.7031304878030136 6.8153355224489645e-28
423.4946002342642 675.1470589448868 676.1462971010833
0.6977967381412655 1.695927632459634 6.851487266979074e-28
426.4979396320484 677.9335583128609 678.9287586390018
0.6897966670312676 1.6713879483532628 6.971306637730841e-28
426.918139759494 678.3226728923854 679.317033259514
0.6886811730973592 1.6679996147104994 6.987430817556367e-28
427.02305581234094 678.4197953402509 679.413935518628
0.6884027772216001 1.6671553271195856 6.9914322759028065e-28
427.02320534096714 678.4198398962733 679.4139800410649
0.6884016887903835 1.6671544099243434 6.991426084019006e-28
427.8761297298557 679.2093888752424 680.2015834699529
0.6861436455492949 1.660314634136

0.39037442225468744 0.9501024591679476 7.846246886243283e-28
565.9941891746954 799.2550406628031 798.3570957552947
0.39015898095654444 0.9496724067806913 7.845076896272208e-28
565.994470394613 799.2551733021442 798.3572281953237
0.3901581513901939 0.9496716446044461 7.845063960436095e-28
567.059489374436 800.1348685548077 799.2287831381858
0.3884280396939926 0.9462169729462535 7.835593215500241e-28
568.1590213166254 801.041433390645 800.12729051894
0.38664584412653075 0.9426731696326005 7.825696215817488e-28
571.958792256781 804.1716366643711 803.2323543553634
0.38055494791032574 0.9305871373979024 7.790864477713299e-28
572.4900915018579 804.6087723454651 803.6663131625714
0.3797108661256253 0.9289171251267831 7.785915763345364e-28
572.6227364539303 804.7178836225603 803.7746443850143
0.37950040484817144 0.9285009466572632 7.784677045830092e-28
572.6230189239765 804.7180171213379 803.774777733222
0.37949959850394094 0.9285002071575491 7.784664310136044e-28
573.691178079985 805.59681242

0.2578190909874636 0.6986008463572108 6.770314781141205e-28
664.7174822555753 878.6229065748013 878.3065543759255
0.257681175593865 0.6983500667784364 6.768841635486576e-28
664.7177054647474 878.62300114227 878.3066493001331
0.25768072800333847 0.6983496990726861 6.768834188735289e-28
665.7189491494569 879.4087807720913 879.1084892272845
0.2565910254810533 0.6963668552822755 6.7571609775537275e-28
666.7521504602835 880.2184667285322 879.9348266887542
0.25546907172876243 0.6943308726680166 6.745108522038515e-28
670.3217191364736 883.0144529757507 882.7889204749463
0.25163199714762813 0.6873680562880586 6.703512074869283e-28
670.8206591758229 883.4049348905528 883.187587904799
0.2511000551170965 0.6864036765990245 6.697702919191767e-28
670.9452169757781 883.5024009349805 883.2870995062964
0.2509674175005048 0.6861632628628209 6.696252770925523e-28
670.9454298215209 883.5024889969375 883.2871879229959
0.25096699661228006 0.6861629232193921 6.696245815151583e-28
671.9302865894731 884.27329

In [39]:
-1.1226882532375977 + param3b.as

387.6985504060269